# Install and Setup Kubeflow on EKS

Based on these instructions:  https://www.kubeflow.org/docs/aws/deploy/install-kubeflow/

# _This Will Take a Few Minutes, Please Be Patient._

In [1]:
%%bash

source ~/.bash_profile

echo $S3_BUCKET
echo $AWS_CLUSTER_NAME
echo $INSTANCE_ROLE_NAME
echo $INSTANCE_PROFILE_ARN

sudo cp kfext/kfctl /usr/local/bin

sagemaker-us-west-2-085964654406
cluster
eksctl-cluster-nodegroup-ng-b0ae5-NodeInstanceRole-1WZRG3O3OHXIB
arn:aws:iam::085964654406:role/eksctl-cluster-nodegroup-ng-b0ae5-NodeInstanceRole-1WZRG3O3OHXIB


In [2]:
%%bash

source ~/.bash_profile

export KF_NAME=${AWS_CLUSTER_NAME}
echo "export KF_NAME=${KF_NAME}" | tee -a ~/.bash_profile

export KF_DIR=$PWD/${KF_NAME}
echo "export KF_DIR=${KF_DIR}" | tee -a ~/.bash_profile

mkdir -p ${KF_DIR}

export KF_NAME=cluster
export KF_DIR=/home/ec2-user/SageMaker/workshop/12_kubeflow/cluster


In [3]:
%%bash

source ~/.bash_profile

cp kfext/v1.0.2.tar.gz ${KF_DIR}
cp kfext/kfctl_aws.v1.0.2.yaml ${KF_DIR}

In [4]:
%%bash

source ~/.bash_profile

export CONFIG_FILE=${KF_DIR}/kfctl_aws.v1.0.2.yaml
echo "export CONFIG_FILE=${CONFIG_FILE}" | tee -a ~/.bash_profile

export CONFIG_FILE=/home/ec2-user/SageMaker/workshop/12_kubeflow/cluster/kfctl_aws.v1.0.2.yaml


In [5]:
%%bash

source ~/.bash_profile

sed -i.bak -e "/region: us-west-2/ a \      enablePodIamPolicy: true" ${CONFIG_FILE}
sed -i.bak -e "s@us-west-2@$AWS_REGION@" ${CONFIG_FILE}
sed -i.bak -e "s@roles:@#roles:@" ${CONFIG_FILE}
sed -i.bak -e "s@- eksctl-kubeflow-aws-nodegroup-ng-a2-NodeInstanceRole-xxxxxxx@#- eksctl-kubeflow-aws-nodegroup-ng-a2-NodeInstanceRole-xxxxxxx@" ${CONFIG_FILE}
sed -i.bak -e "s@eksctl-kubeflow-aws-nodegroup-ng-a2-NodeInstanceRole-xxxxxxx@$INSTANCE_ROLE_NAME@" ${CONFIG_FILE}
sed -i.bak -e 's/kubeflow-aws/'"$AWS_CLUSTER_NAME"'/' ${CONFIG_FILE}

In [6]:
%%bash

source ~/.bash_profile

cd ${KF_DIR}

rm -rf kustomize
rm -rf .cache

kfctl build -V -f ${CONFIG_FILE}

cd ${KF_DIR}

kfctl apply -V -f ${CONFIG_FILE}

kubectl delete all -l app=spartakus --namespace=kubeflow

kubectl config set-context --current --namespace=kubeflow

customresourcedefinition.apiextensions.k8s.io/adapters.config.istio.io created
customresourcedefinition.apiextensions.k8s.io/apikeys.config.istio.io created
customresourcedefinition.apiextensions.k8s.io/attributemanifests.config.istio.io created
customresourcedefinition.apiextensions.k8s.io/authorizations.config.istio.io created
customresourcedefinition.apiextensions.k8s.io/bypasses.config.istio.io created
customresourcedefinition.apiextensions.k8s.io/certificates.certmanager.k8s.io created
customresourcedefinition.apiextensions.k8s.io/challenges.certmanager.k8s.io created
customresourcedefinition.apiextensions.k8s.io/checknothings.config.istio.io created
customresourcedefinition.apiextensions.k8s.io/circonuses.config.istio.io created
customresourcedefinition.apiextensions.k8s.io/cloudwatches.config.istio.io created
customresourcedefinition.apiextensions.k8s.io/clusterissuers.certmanager.k8s.io created
customresourcedefinition.apiextensions.k8s.io/clusterrbacconfigs.rbac.istio.io creat

time="2020-09-26T21:39:01Z" level=info msg="No name specified in KfDef.Metadata.Name; defaulting to cluster based on location of config file: /home/ec2-user/SageMaker/workshop/12_kubeflow/cluster/kfctl_aws.v1.0.2.yaml." filename="coordinator/coordinator.go:202"
time="2020-09-26T21:39:01Z" level=info msg="Running `eksctl version` ..." filename="utils/awsutil.go:71"
time="2020-09-26T21:39:01Z" level=info msg="eksctl version: 0.28.1" filename="utils/awsutil.go:88"
time="2020-09-26T21:39:01Z" level=info msg="\n****************************************************************\nNotice anonymous usage reporting enabled using spartakus\nTo disable it\nIf you have already deployed it run the following commands:\n  cd $(pwd)\n  kubectl -n ${K8S_NAMESPACE} delete deploy -l app=spartakus\n\nFor more info: https://www.kubeflow.org/docs/other-guides/usage-reporting/\n****************************************************************\n\n" filename="coordinator/coordinator.go:120"
time="2020-09-26T21:39:

## Wait for resource to become available
Monitor changes by running kubectl get all namespaces command.


In [7]:
!kubectl -n kubeflow get all

NAME                                                               READY   STATUS              RESTARTS   AGE
pod/admission-webhook-bootstrap-stateful-set-0                     1/1     Running             0          11s
pod/admission-webhook-deployment-59bc556b94-2rttd                  0/1     ContainerCreating   0          1s
pod/admission-webhook-deployment-59bc556b94-j9n4l                  0/1     Terminating         0          11s
pod/alb-ingress-controller-798fbd8c96-jmc2n                        0/1     ContainerCreating   0          3s
pod/application-controller-stateful-set-0                          1/1     Running             0          70s
pod/argo-ui-5f845464d7-tjr7g                                       0/1     Running             0          11s
pod/centraldashboard-d5c6d6bf-zk7fv                                0/1     ContainerCreating   0          11s
pod/jupyter-web-app-deployment-544b7d5684-427t6                    0/1     ContainerCreating   0          10s
pod/katib-co

In [8]:
!kubectl -n istio-system get all

NAME                                          READY   STATUS      RESTARTS   AGE
pod/cluster-local-gateway-78f6cbff8d-28gsr    1/1     Running     0          73s
pod/grafana-68bcfd88b6-2djq8                  1/1     Running     0          75s
pod/istio-citadel-7dd6877d4d-qwgpt            1/1     Running     0          75s
pod/istio-cleanup-secrets-1.1.6-nq7hz         0/1     Completed   0          75s
pod/istio-egressgateway-7c888bd9b9-nvc57      0/1     Running     0          75s
pod/istio-galley-5bc58d7c89-mzv6r             1/1     Running     0          75s
pod/istio-grafana-post-install-1.1.6-kfmv2    0/1     Completed   0          75s
pod/istio-ingressgateway-866fb99878-pf6s4     1/1     Running     0          75s
pod/istio-pilot-67f9bd57b-v9bkn               2/2     Running     0          75s
pod/istio-policy-749ff546dd-48nq8             2/2     Running     2          75s
pod/istio-security-post-install-1.1.6-dtglt   0/1     Completed   0          75s
pod/istio-sidecar-injector-c

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();